## Loading LIbraries

In [60]:
import pandas as pd
import seaborn as sns
from elo import rate_1vs1
import numpy as np
import scipy

## Loading Match Data

In [7]:
matches=pd.read_csv('MatchData.csv',low_memory=False)  ## IDk what low memory does but it works



### Getting List of all teams

In [8]:

## List of all teams
teamset=set()
for team in matches['HomeTeam']:
    teamset.add(team)
for team in matches['AwayTeam']:
    teamset.add(team)
print(teamset)

{nan, 'Granada', 'Valencia', 'Malaga', 'Eibar', 'Girona', 'Espanol', 'Getafe', 'Numancia', 'La Coruna', 'Ath Madrid', 'Las Palmas', 'Recreativo', 'Cordoba', 'Huesca', 'Alaves', 'Santander', 'Extremadura', 'Ath Bilbao', 'Zaragoza', 'Villarreal', 'Sociedad', 'Elche', 'Levante', 'Betis', 'Celta', 'Compostela', 'Real Madrid', 'Lerida', 'Logrones', 'Barcelona', 'Sp Gijon', 'Vallecano', 'Merida', 'Hercules', 'Almeria', 'Osasuna', 'Gimnastic', 'Valladolid', 'Mallorca', 'Murcia', 'Tenerife', 'Albacete', 'Oviedo', 'Sevilla', 'Cadiz', 'Leganes', 'Salamanca', 'Xerez', 'Villareal'}


TeamData of each and every team( We will add fields as we go on)

In [39]:
teamsdata={}
for team in teamset:
    teamdata={}
    teamdata['matches_played']=0
    teamdata['won']=0
    teamdata['tie']=0
    teamdata['elo']=1200  #Default rating
    teamsdata[team]=teamdata

In [10]:
matches.describe()

,Unnamed: 0,FTHG,FTAG,Season,HTHG,HTAG,GBH,GBD,GBA,IWH,...,AvgC<2.5,AHCh,B365CAHH,B365CAHA,PCAHH,PCAHA,MaxCAHH,MaxCAHA,AvgCAHH,AvgCAHA
count,11051.000000,10355.000000,10355.000000,11051.000000,9593.000000,9593.000000,4729.000000,4729.000000,4729.000000,7509.000000,...,423.000000,423.000000,423.000000,423.000000,423.000000,423.000000,423.000000,423.000000,423.000000,423.000000
mean,208.964890,1.568904,1.109512,2005.249842,0.693214,0.483061,2.355394,3.598336,4.379975,2.438172,...,1.863664,-0.316194,1.955792,1.958629,1.959669,1.962530,2.018463,2.018936,1.937470,1.940449
std,129.742154,1.318418,1.117745,8.097132,0.843592,0.696223,1.424818,0.942945,3.087454,1.665292,...,0.455824,0.740895,0.102628,0.104099,0.103082,0.103994,0.104796,0.109858,0.097723,0.097881
min,0.000000,0.000000,0.000000,1993.000000,0.000000,0.000000,1.050000,2.350000,1.120000,1.030000,...,1.290000,-2.750000,1.680000,1.670000,1.700000,1.750000,1.780000,1.800000,1.710000,1.750000
25%,100.000000,1.000000,0.000000,1998.000000,0.000000,0.000000,1.700000,3.200000,2.750000,1.650000,...,1.550000,-0.750000,1.880000,1.880000,1.880000,1.880000,1.940000,1.930000,1.860000,1.860000
50%,203.000000,1.000000,1.000000,2005.000000,0.000000,0.000000,2.050000,3.300000,3.500000,2.000000,...,1.740000,-0.250000,1.960000,1.970000,1.950000,1.970000,2.010000,2.020000,1.930000,1.940000
75%,306.000000,2.000000,2.000000,2012.000000,1.000000,1.000000,2.450000,3.500000,4.800000,2.500000,...,2.040000,0.000000,2.040000,2.040000,2.040000,2.045000,2.100000,2.105000,2.015000,2.020000
max,551.000000,10.000000,8.000000,2020.000000,6.000000,6.000000,17.000000,11.000000,34.000000,20.000000,...,4.130000,1.750000,2.160000,2.190000,2.200000,2.290000,2.260000,2.300000,2.150000,2.210000


In [46]:
for index, row in matches.iterrows(): 
    ht=row['HomeTeam']
    teamsdata[ht]['matches_played']+=1
    
    at=row["AwayTeam"]
#     print(ht ,at)
    teamsdata[at]['matches_played']+=1
    result=row['FTR']
#     print(result)
    if(result=='D'):
#         print("Tie")
        teamsdata[ht]['tie']+=1
        teamsdata[at]['tie']+=1
        wer=teamsdata[ht]['elo']
        ler=teamsdata[at]['elo']
        nwer , nler = rate_1vs1(wer,ler,drawn=True)
        teamsdata[ht]['elo']=nwer
        teamsdata[at]['elo']=nler
    else:
        
        winner=ht if result=='H' else at
        loser=at if result=='H' else ht
#         print("Winner",winner)
        teamsdata[winner]['won']+=1
        ## Updata elo ratings
        wer=teamsdata[winner]['elo']
        ler=teamsdata[loser]['elo']
        nwer , nler = rate_1vs1(wer,ler)
        teamsdata[winner]['elo']=nwer
        teamsdata[loser]['elo']=nler
        

teamsdata

{'Alaves': {'elo': 1200.1287207782807,
  'matches_played': 770,
  'tie': 172,
  'won': 258},
 'Albacete': {'elo': 1153.366268514086,
  'matches_played': 369,
  'tie': 116,
  'won': 94},
 'Almeria': {'elo': 1143.3352592542162,
  'matches_played': 456,
  'tie': 112,
  'won': 124},
 'Ath Bilbao': {'elo': 1252.4961550183923,
  'matches_played': 2053,
  'tie': 572,
  'won': 761},
 'Ath Madrid': {'elo': 1415.9123501465199,
  'matches_played': 1901,
  'tie': 468,
  'won': 897},
 'Barcelona': {'elo': 1503.873783700005,
  'matches_played': 2053,
  'tie': 414,
  'won': 1315},
 'Betis': {'elo': 1224.7134778285815,
  'matches_played': 1676,
  'tie': 472,
  'won': 574},
 'Cadiz': {'elo': 1182.5187406069622,
  'matches_played': 86,
  'tie': 26,
  'won': 20},
 'Celta': {'elo': 1206.6617077775104,
  'matches_played': 1623,
  'tie': 445,
  'won': 562},
 'Compostela': {'elo': 1190.1820536264206,
  'matches_played': 320,
  'tie': 90,
  'won': 104},
 'Cordoba': {'elo': 1132.1541546490569,
  'matches_playe

In [50]:
teamstable=pd.DataFrame.from_dict(teamsdata,orient='Index')
teamstable['Team']=teamstable.index
teamstable.sort_values(by=['elo'],inplace=True,ascending=False)
topteams=teamstable.head(8)
topteams

,matches_played,won,tie,elo,Team
Barcelona,2053,1315,414,1503.873784,Barcelona
Real Madrid,2056,1292,392,1461.179549,Real Madrid
Ath Madrid,1901,897,468,1415.912350,Ath Madrid
Sevilla,1824,778,440,1347.593982,Sevilla
Villarreal,1430,602,372,1297.289494,Villarreal
Valencia,2054,948,502,1292.175058,Valencia
Sociedad,1825,662,494,1272.018088,Sociedad
Getafe,1126,380,290,1266.070782,Getafe


In [52]:
sns.catplot(topteams, x="Team",y="elo")

/home/gagan/.local/lib/python3.6/site-packages/seaborn/_decorators.py:43: FutureWarning: Pass the following variable as a keyword arg: x. From version 0.12, the only valid positional argument will be `data`, and passing other arguments without an explicit keyword will result in an error or misinterpretation.
  FutureWarning


ValueError: Could not interpret input 'elo'

In [111]:
players=pd.read_csv('player_scraper/playerdata.csv',low_memory=False)

In [11]:
players.head()

,name,goals_per90,assists_per90,goals_assists_pens_per90,goals_pens_per90,goals,shots_total_per90,shots_on_target_per90,shots_free_kicks,pens_made,...,dribbles_vs,pressures,pressure_regains,blocks,blocked_shots,dribbles_completed,carry_distance,fouls,aerials_won,ball_recoveries
0,Andriy Lunin,"['0.00', None, None]","['0.00', None, None]","['0.00', None, None]","['0.00', None, None]",NaN,NaN,NaN,NaN,NaN,...,"[None, '0', None]","[None, '0', None]","[None, '0', None]","[None, '1', None]","[None, '0', None]","[None, '0', None]","[None, '249', None]","0,0,1","[None, '0', None]","[None, '21', None]"
1,Víctor Chust,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Thibaut Courtois,"0.00,0.00,0.00","0.00,0.00,0.00","0.00,0.00,0.00","0.00,0.00,0.00","0,0,0","0.04,0.03,0.00","0.00,0.03,0.00","0,0,0","0,0,0",...,"3,2,0","3,1,1","0,0,0","0,0,0","0,0,0","1,0,0","2317,3238,564","0,0,0","0,1,0","118,161,13"
3,Diego Altube,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Alfonso Espino,"0.00,0.03,0.00","0.21,0.06,0.00","0.21,0.09,0.00","0.00,0.03,0.00","0,1,0","[None, '0.49', '0.40']","[None, '0.09', '0.00']","[None, None, '0']","0,0,0",...,"[None, None, '15']","[None, None, '87']","[None, None, '23']","[None, None, '15']","[None, None, '0']","[None, None, '3']","[None, None, '470']","[None, '58', '6']","[None, None, '3']","[None, None, '45']"


In [112]:
# We will compute three seasons average of all stats
def calcaverage(listt):
    if(pd.isna(listt)):
        return
    else:
        listt=listt.strip('][').split(',')
        cc=0
        val=0
        for x in listt:
            x=x.strip('\' ')
            
            if(x!="None"):
                cc+=1
                val=val+float(x)
        if(cc==0):
            return
        return (val/cc)
for c in columns:
    if(c=="name"):
        continue
    players[c]=players[c].apply(calcaverage)
    
#     players[c]=scipy.stats.norm.sf(abs(scipy.stats.zscore((players[c].to_numpy()),nan_policy='omit')))*200
    




In [119]:
currentcol=""
playerscores=pd.DataFrame()
def calcscore(x):
    return scipy.stats.percentileofscore(players[c].to_numpy(), x , kind='rank')
for c in columns:
    if(c=="name"):
        playerscores[c]=players[c]
        continue
    playerscores[c]=players[c].apply(calcscore)
#     players[c]=scipy.stats.norm.sf(abs(scipy.stats.zscore((players[c].to_numpy()),nan_policy='omit')))*200
    




In [120]:


temp=playerscores.sort_values(by="goals_assists_pens_per90",ascending=False)
temp.head()
# temp[temp['goals']>10]

,name,goals_per90,assists_per90,goals_assists_pens_per90,goals_pens_per90,goals,shots_total_per90,shots_on_target_per90,shots_free_kicks,pens_made,...,dribbles_vs,pressures,pressure_regains,blocks,blocked_shots,dribbles_completed,carry_distance,fouls,aerials_won,ball_recoveries
160,Antonio Cortés,91.187739,15.900383,91.187739,91.187739,58.716475,73.563218,79.693487,28.544061,33.141762,...,14.080460,25.478927,25.383142,18.869732,13.122605,38.218391,18.007663,45.114943,31.034483,15.134100
163,Roberto López,88.505747,90.996169,90.996169,88.697318,33.812261,51.915709,60.919540,65.038314,33.141762,...,10.919540,13.601533,14.176245,15.038314,13.122605,17.528736,8.045977,7.375479,25.670498,12.260536
274,Denis Cheryshev,55.842912,91.187739,90.804598,57.279693,42.911877,48.850575,50.191571,28.544061,33.141762,...,67.528736,71.647510,67.528736,66.858238,50.862069,63.122605,56.321839,72.126437,54.406130,48.084291
296,Luis Suárez,90.613027,89.463602,90.613027,90.613027,79.885057,79.310345,79.310345,73.754789,76.724138,...,37.068966,65.325670,63.984674,47.892720,46.647510,81.896552,64.559387,55.076628,32.758621,43.295019
275,Ansu Fati,90.804598,50.478927,90.421456,90.804598,72.796935,73.180077,77.969349,28.544061,33.141762,...,40.900383,49.042146,50.287356,43.582375,31.800766,70.785441,63.601533,37.835249,28.639847,28.065134


In [179]:

def givescorehelper(features,weightage):
    val=0
    cc=0
    for i in range(0, len(features)):
        if(pd.isna(features[i])):
            continue
        val+=features[i]*weightage[i]
        cc+=weightage[i]
    if(cc==0):
        return
    return val/cc


finalscores=pd.DataFrame()
finalscores['name']=players['name']
finalscores
maxscore=-1
def roundoff(x):
    global maxscore
    fact=100/maxscore
    return round(x*fact)
def  givescore(stylename , featurenames ,weightage):
    maxscore=-1
    for i,row in playerscores.iterrows():
        name=row['name']
        features=[]
        for x in featurenames:
            features.append(playerscores.loc[i,x])
        
        score=givescorehelper(features,weightage)
        if(score==None):
            continue
        score=max(maxscore,score)
        finalscores.loc[i,stylename]=score
    finalscores[stylename]=finalscores[stylename].apply(roundoff())
        
    
# this is where we give scores
givescore('striker',['goals_assists_pens_per90','shots_on_target_per90','goals'],[2,1,1])  
givescore('midfeilder',['carry_distance','passes_completed'],[1,1])

TypeError: roundoff() missing 1 required positional argument: 'x'

In [176]:
finalscores.sort_values(by="midfeilder",ascending=False)

,name,striker,midfeilder
284,Gerard Piqué,40.900383,88.314176
143,Sergio Canales,73.826628,87.452107
213,Daniel Parejo,58.477011,87.452107
440,Toni Kroos,56.872605,87.164751
422,Jesús Navas,52.035441,87.068966
...,...,...,...
480,César,NaN,NaN
481,Luis Castillo,NaN,NaN
482,Jonatan Carmona Alamo,10.919540,NaN
499,Diego Bri,10.919540,NaN


In [177]:
finalscores.sort_values(by="striker",ascending=False)

,name,striker,midfeilder
288,Lionel Messi,85.153257,86.973180
296,Luis Suárez,85.105364,64.080460
424,Luuk de Jong,83.189655,37.212644
20,Roger Martí,83.069923,40.325670
275,Ansu Fati,82.902299,55.028736
...,...,...,...
476,Kevin Omoruyi,NaN,NaN
478,Ignacio Pastor,NaN,NaN
480,César,NaN,NaN
481,Luis Castillo,NaN,NaN
